In [ ]:
### Importing the libaries for image processing

import cv2
import numpy
import tensorflow
from PIL import Image
import os

In [7]:
class ImageHandler:

    def load_image(self, path):
        self.path = path

        try:
            img = Image.open(self.path)
            print("Import Successful")
        except FileNotFoundError:
            print(f"Error: Image not found")
        except Exception as e:
            print(f"An error occurred: {e}")

    def save_image(self, img, path):
        pass

    def resize_image(self, img, max_dim):
        pass

    def to_rgb(self, img):
        pass
    
    def to_bgr(self, img):
        pass

    def validate_format(self, path):
        pass

    def preprocess_for_model(self, img):  # Brings everythhing together
        pass


In [21]:
img = ImageHandler()

import sys
import os

# Automatically get the project root
project_root = os.path.abspath(os.path.join(os.getcwd(), "..", ".."))
sys.path.append(project_root)

print(project_root)

img.load_image(path)

/home/wayne/Github/PrivacyEnc
An error occurred: 'list' object has no attribute 'read'
